In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Activation, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D, Conv2D, Input, Concatenate, Add
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from keras.utils import np_utils
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
#from keras.layers.advanced_activations import ELU
from keras.initializers import glorot_uniform
from keras import backend as K
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
#from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import save_model


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
import h5py
import numpy as np
import os,random
from tensorflow.keras.layers import Input,Reshape,ZeroPadding2D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout
from tensorflow.keras import layers
import tensorflow.keras.models as Model
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

In [3]:
LENGTH = 500
NB_CLASSES = 4 # number of outputs = number of classes
VERBOSE = 1
BATCH_SIZE = 10
NB_EPOCH = 10
target_names = ['Non-MO', 'MO']
file_short = "D:/BMO/Main/savedModels/_short.h5"
file_long = "D:/BMO/Main/savedModels/_long.h5"
file_best = "D:/BMO/Main/savedModels/_best.h5"

In [4]:
%%time

print(".........Loading MO (2-state) dataset..........")

# sep separates each column and header = none means it is going to read from row 1
re2psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/BPSK-2/real_snr00.csv',sep=',', header=None) 
im2psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/BPSK-2/im_snr00.csv',sep=',', header=None)

# sep separates each column and header = none means it is going to read from row 1
re4psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/QPSK-2/real_snr00.csv',sep=',', header=None) 
im4psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/QPSK-2/im_snr00.csv',sep=',', header=None)

# sep separates each column and header = none means it is going to read from row 1
re16qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/16-QAM-2/real_snr00.csv',sep=',', header=None) 
im16qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/16-QAM-2/im_snr00.csv',sep=',', header=None)

# sep separates each column and header = none means it is going to read from row 1
re64qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/64-QAM-2/real_snr00.csv',sep=',', header=None) 
im64qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/SNR-based/64-QAM-2/im_snr00.csv',sep=',', header=None)

#merge two dataframes into one
df2psk_2st = re2psk_2st + im2psk_2st * 1j
df2psk_2st['Mod'] = 0 # 0 = BPSK 2-state
#merge two dataframes into one
df4psk_2st = re4psk_2st + im4psk_2st * 1j
df4psk_2st['Mod'] = 1 # 1 = QPSK 2-state
#merge two dataframes into one
df16qam_2st = re16qam_2st + im16qam_2st * 1j
df16qam_2st['Mod'] = 2 # 2 = 16-QAM 2-state
#merge two dataframes into one
df64qam_2st = re64qam_2st + im64qam_2st * 1j
df64qam_2st['Mod'] = 3 # 3 = 64-QAM 2-state

# combine all
df_2st = df2psk_2st.append(df4psk_2st)
df_2st = df_2st.append(df16qam_2st)
df_2st = df_2st.append(df64qam_2st)

df_2st = df_2st.sample(frac = 1)
#df_2st = df_2st.iloc[:5000]
print('df_2st Shape = ', df_2st.shape)

.........Loading MO (2-state) dataset..........
df_2st Shape =  (28000, 2049)
CPU times: total: 45.1 s
Wall time: 44.7 s


In [5]:
print("..........Combining all data..........")

# combine all
#df_all = dfmc.append(df_2st)
df_all = df_2st.sample(frac = 1)
print('df_all Shape = ', df_all.shape)
print("..........Shuffling done..........")

# reducing samples to tune only
df_all = df_all.iloc[:6384]
print('df_all Size = ', df_all.size)
print('Dimension = ', df_all.ndim)
print('Combined Shape = ', df_all.shape)

..........Combining all data..........
df_all Shape =  (28000, 2049)
..........Shuffling done..........
df_all Size =  13080816
Dimension =  2
Combined Shape =  (6384, 2049)


In [6]:
# Separating X and y
Y = df_all['Mod'] # 1D targer vector
X = df_all.drop(columns='Mod')
print(Y.shape)
Y = np_utils.to_categorical(Y, NB_CLASSES)
INPUT_SHAPE = (LENGTH,1)
    
X.drop(X.iloc[:, LENGTH:2048], inplace = True, axis = 1)

X = np.expand_dims(X, -1)

n_examples = X.shape[0]
# n_train = int(n_examples * 0.9877) 
n_train = int(n_examples * 0.8)  
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)  #Randomly select training sample subscript
test_idx = list(set(range(0,n_examples))-set(train_idx)) #Test sample index
#print(train_idx, test_idx)
X_train = X[train_idx]  #training samples
X_test =  X[test_idx]  #testing samples
Y_train = Y[train_idx]
Y_test = Y[test_idx]
print("X_train:",X_train.shape)
print("Y_train:",Y_train.shape)
print("X_test:",X_test.shape)
print("Y_test:",Y_test.shape)

(6384,)
X_train: (5107, 500, 1)
Y_train: (5107, 4)
X_test: (1277, 500, 1)
Y_test: (1277, 4)


In [7]:
print(X_train.shape[1:])
# Input(in_shp)
classes = target_names

(500, 1)


In [8]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights 
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.2):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="ELU"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
    def get_config(self):
        cfg = super().get_config()
#         cfg.update({
#             'att': self.MultiHeadSelfAttention(1024, 128),
#             'ffn': self.tf.keras.Sequential(
#             [layers.Dense(256, activation="relu"), layers.Dense(1024),]
#         ),
#             'layernorm1': self.layers.LayerNormalization(epsilon=1e-6),
#             'layernorm2': self.layers.LayerNormalization(epsilon=1e-6),
#             'dropout1': self.Dropout(0.2),
#             'dropout2': self.Dropout(0.2),
#         })
        return cfg    
    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
def proposed_model(X_train,classes):

    embed_dim = LENGTH  # Embedding size for each token
    num_heads = 500  # Number of attention heads
    ff_dim = 1000  # Hidden layer size in feed forward network inside transformer

#     inputs = layers.Input(shape=(maxlen,))
#     embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
#     x = embedding_layer(inputs)
#     mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:2"], cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
#     with mirrored_strategy.scope():

    in_shp = X_train.shape[1:]   #Dimensions of each sample

    #input layer
    X_input = Input(in_shp)
    X = Reshape([1,LENGTH], input_shape=in_shp)(X_input)

    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(X)
    #x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.AlphaDropout(0.3)(x)
    x = layers.Dense(NB_CLASSES, activation="softmax")(x)
    return tf.keras.models.Model(inputs=X_input, outputs=x)     
    
model = proposed_model(X_train,classes)    
model.compile(loss='categorical_crossentropy', optimizer=tfa.optimizers.LazyAdam(),metrics=['accuracy'])
#model.load_weights('D:\BMO\Main\trafo_model')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 1)]          0         
                                                                 
 reshape (Reshape)           (None, 1, 500)            0         
                                                                 
 transformer_block (Transfor  (None, None, 500)        2005500   
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 500)              0         
 lobalAveragePooling1D)                                          
                                                                 
 alpha_dropout (AlphaDropout  (None, 500)              0         
 )                                                               
                                                             

In [9]:
%%time
filepath = "D:/BMO/Main/savedModels/_best.h5"
history = model.fit(X_train,
    Y_train,
    batch_size=10,
    epochs=10,
    verbose=1,
    shuffle=True,
#     validation_data=(X_test, Y_test),
    validation_split = 0.1,
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
    ])

Epoch 1/10
458/460 [============================>.] - ETA: 0s - loss: 1.8829 - accuracy: 0.2712
Epoch 1: val_loss improved from inf to 1.64381, saving model to D:/BMO/Main/savedModels\_best.h5
460/460 [==============================] - 9s 13ms/step - loss: 1.8819 - accuracy: 0.2715 - val_loss: 1.6438 - val_accuracy: 0.2779
Epoch 2/10
459/460 [============================>.] - ETA: 0s - loss: 1.4255 - accuracy: 0.3381
Epoch 2: val_loss improved from 1.64381 to 1.58086, saving model to D:/BMO/Main/savedModels\_best.h5
460/460 [==============================] - 5s 11ms/step - loss: 1.4257 - accuracy: 0.3383 - val_loss: 1.5809 - val_accuracy: 0.3209
Epoch 3/10
458/460 [============================>.] - ETA: 0s - loss: 1.3314 - accuracy: 0.3930
Epoch 3: val_loss improved from 1.58086 to 1.54332, saving model to D:/BMO/Main/savedModels\_best.h5
460/460 [==============================] - 5s 11ms/step - loss: 1.3319 - accuracy: 0.3925 - val_loss: 1.5433 - val_accuracy: 0.3190
Epoch 4/10
459/46

In [17]:
Y_pred = model.predict(X_test)
    
Y_pred1=np.argmax(Y_pred, axis=1)
Y_test1=np.argmax(Y_test, axis=1)
print(classification_report(Y_test1,Y_pred1))

40/40 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

           0       0.26      0.16      0.19       290
           1       0.33      0.36      0.34       337
           2       0.31      0.36      0.33       309
           3       0.37      0.41      0.39       341

    accuracy                           0.33      1277
   macro avg       0.32      0.32      0.32      1277
weighted avg       0.32      0.33      0.32      1277

